# 구매전환율 기준 마련 후 구매 전환율 계산.

## 첫 구매 전환율(첫 번째 구매한 사람/ 가입일자를 가지고 있는 가입자 ID 수)

### 분모
1. 가입일자가 없는 유저ID를 제거한다.(모든 데이터가 가지고 있었음)
  
  → 가입일자가 없는 유저는 구매를 할 수 없고 측정할 수 없어서 가입일자가 없는 유저는 제외했다.
2. 유저 ID를 중복제거를 한 데이터의 수를 구한다.
  
  → 분모에서 유저ID로 중복제거를 한 것은 말 같은 사람이 여러번 살 경우를 배제하기 위해서이다.


### 분자
1. 처음 테이블의 데이터에서 구매일자가 없는 데이터를 제거한다.
  
  → 구매일자가 없는 데이터는 구매를 하지 않은 데이터이기 때문에 삭제한다.
2. 구매 날짜를 최신순으로 정렬한 후 유저 ID로 중복제거를 한 데이터의 수를 구한다.(중복제거 시 마지막 행만 남도록 한다.)
  
  → 구매 날짜를 최신순으로 정렬한 것은 유저 ID로 중복제거시 가장 처음 구입한 날짜를 구하기 위해서이다.
  
※ 추가로 첫 구매까지 몇 일이 걸리는지 알아보았다.


### 가입 후 첫 구매 평균일자
1. 구매일에서 가입을을 뺀다.
2. joined_date 의 데이터와 paid_date의 데이터 타입이 달라서 빼기가 되질 않아
values로 값을 뽑고 astype('datetime64[D]')로 타입을 변경한 다음 pd.Series로 만들어 두 데이터의 타입을 맞추고 날짜를 뺐다.



### 결과
5621 / 15512 로 첫 구매 전환율은 36.24%가 나왔으며 첫 구매까지 평균 ##일이 걸렸다.


### 해석
가입자 수가 적어서 그럴수도 있지만 그럼에도 불구하고 첫 구매 전환율이 36.24%로 매우 높으므로 신규 가입자 수를 많이 늘릴 수 있는 것이 필요하다.


## 재구매 전환율(두 번째 구매한 사람/ 첫 번째 구매한 사람)


### 분모
1. 첫 번째 구매 전환율의 분자를 사용한다.
  
  → 첫 번째 구매 후 다음 구매를 몇 % 구매했는가를 보기 위해서
  
  
  
### 분자
1. 처음 테이블의 데이터에서 구매일자가 없는 데이터를 제거한다.

  → 구매일자가 없는 데이터는 구매를 하지 않은 데이터이기 때문에 삭제한다.
2. 구매 날짜를 최신순으로 정렬한 후 유저 ID로 제일 중복된 유저 ID의 맨 마지막 일자의 유저 ID를 삭제한다.
  
  → 첫 번째 구매내역만 삭제하기 위해서 이다.
3. 구매 날짜가 최신순으로 되어 있으므로 유저 ID로 맨 중복의 맨 마지막만 남겨두고 모두 삭제한다.
  
  → 두 번째 구매내역만 삭제하기 위해서 이다.
  
  

### 결과
5621 / 5621 로 재구매 전환율은 100% 이다.



### 해석
구매 후 재구매는 무조껀 일어난다고 할 수 있다.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt

import warnings
warnings.filterwarnings('ignore')

In [42]:
total_df = pd.DataFrame()
payment_df = pd.read_csv('./payment_window.csv', encoding='CP949')
bra_df = pd.read_csv('./bra_size_window.csv', encoding='CP949')
user_df = pd.read_csv('./user_window.csv', encoding='CP949')
pay_bra_df = pd.merge(payment_df, bra_df, how='outer', on='user_idx')
total_df = pd.merge(pay_bra_df, user_df, on='user_idx')
drop_cols = ['Unnamed: 0_x', 'Unnamed: 0', 'Unnamed: 0_y']
total_df.drop(drop_cols, axis=1, inplace=True)
total_df.reset_index(drop=True, inplace=True)

total_df['paid_date'] = pd.to_datetime(total_df['paid_date'])
total_df['joined_date'] = pd.to_datetime(total_df['joined_date'])
total_df['created_date'] = pd.to_datetime(total_df['created_date'])

In [43]:
print(payment_df.shape)
display(payment_df.head())
display(payment_df.describe().T)
print(payment_df.info())

(7308, 7)


,Unnamed: 0,user_idx,subscription_type,cart_discount_price,order_shipping_price,order_total_price,paid_date
0,1,0973f573635c1b55a58f1d221f992bfeb0b3e007a93efb...,구독,5100,3000,12900,2020-10-21
1,2,305181c0babb5077b46ccb298a41469729a6760b19e44d...,구독,5100,3000,12900,2020-10-09
2,3,5fa450bdb2e831617f0132293d63638e0f93f1f62af4ca...,구독,5100,3000,12900,2020-10-11
3,4,85649b91d8ac8d2e84faa6c54032254356fe02c69d5fa0...,구독,5100,3000,12900,2020-10-14
4,5,05afd1abc70fce9afaa18482a0bb9c6cb082e65b1e0ab4...,구독,5100,3000,12900,2020-10-21


,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,7308.0,3654.500000,2109.782216,1.0,1827.75,3654.5,5481.25,7308.0
cart_discount_price,7308.0,3267.939245,2729.677090,0.0,0.00,5100.0,5100.00,15000.0
order_shipping_price,7308.0,2970.443350,296.324772,0.0,3000.00,3000.0,3000.00,3000.0
order_total_price,7308.0,14658.949097,3071.579009,0.0,12900.00,12900.0,18000.00,27900.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7308 entries, 0 to 7307
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            7308 non-null   int64 
 1   user_idx              7308 non-null   object
 2   subscription_type     7277 non-null   object
 3   cart_discount_price   7308 non-null   int64 
 4   order_shipping_price  7308 non-null   int64 
 5   order_total_price     7308 non-null   int64 
 6   paid_date             7308 non-null   object
dtypes: int64(4), object(3)
memory usage: 399.8+ KB
None


In [5]:
total_df.columns

Index(['user_idx', 'subscription_type', 'cart_discount_price',
       'order_shipping_price', 'order_total_price', 'paid_date', 'size',
       'created_date', 'status', 'third_party_join_type', 'joined_date'],
      dtype='object')

# 첫 구매 전환율(첫 번째 구매한 사람/ 가입일자를 가지고 있는 가입자 ID 수)

In [89]:
total_df.shape

# 가입날짜 없는 데이터 거르기(가입날짜 없는 데이터 없음)
total_join_df = total_df[total_df['joined_date'].notnull()]
total_join_df.shape

# 분모
total_dupli_df = total_df.drop_duplicates(subset=['user_idx'], keep='last')
total_dupli_df.shape


# 분자
total_paid_df = total_df[total_df['paid_date'].notnull()]
total_paid_df.shape

total_paid_sort_df = total_paid_df.sort_values(by='paid_date', ascending=False)
total_paid_sort_df.shape

total_paid_sort_dupli_df = total_paid_sort_df.drop_duplicates(subset=['user_idx'], keep='last')
total_paid_sort_dupli_df.shape


# 첫 구매까지 걸리는 평균 기간
print("첫 구매 전환율 : ",total_paid_sort_dupli_df.shape[0]/total_dupli_df.shape[0] * 100)
print("평균 첫 구매까지 걸리는 기간 : ",pd.Series(total_paid_sort_dupli_df['paid_date'].values.astype('datetime64[D]')) - \
      pd.Series(total_paid_sort_dupli_df['joined_date'].values.astype('datetime64[D]')))

첫 구매 전환율 :  36.23646209386281
평균 첫 구매까지 걸리는 기간 :  0        0 days
1      118 days
2        0 days
3        1 days
4        0 days
         ...   
5616    36 days
5617   367 days
5618    31 days
5619     0 days
5620   153 days
Length: 5621, dtype: timedelta64[ns]


In [94]:
sum_date = pd.Series(total_paid_sort_dupli_df['paid_date'].values.astype('datetime64[D]')) - \
           pd.Series(total_paid_sort_dupli_df['joined_date'].values.astype('datetime64[D]'))

In [72]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17199 entries, 0 to 17198
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype                                
---  ------                 --------------  -----                                
 0   user_idx               17199 non-null  object                               
 1   subscription_type      7277 non-null   object                               
 2   cart_discount_price    7308 non-null   float64                              
 3   order_shipping_price   7308 non-null   float64                              
 4   order_total_price      7308 non-null   float64                              
 5   paid_date              7308 non-null   datetime64[ns]                       
 6   size                   16588 non-null  object                               
 7   created_date           16602 non-null  datetime64[ns, pytz.FixedOffset(540)]
 8   status                 17199 non-null  int64                      

In [85]:
total_paid_sort_dupli_df['paid_date']

6561   2020-10-31
6558   2020-10-31
5338   2020-10-31
2559   2020-10-31
6545   2020-10-31
          ...    
2098   2020-09-10
79     2020-09-09
6852   2020-09-09
5256   2020-09-07
1675   2020-08-31
Name: paid_date, Length: 5621, dtype: datetime64[ns]

In [46]:
total_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17199 entries, 0 to 17198
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype                                
---  ------                 --------------  -----                                
 0   user_idx               17199 non-null  object                               
 1   subscription_type      7277 non-null   object                               
 2   cart_discount_price    7308 non-null   float64                              
 3   order_shipping_price   7308 non-null   float64                              
 4   order_total_price      7308 non-null   float64                              
 5   paid_date              7308 non-null   datetime64[ns]                       
 6   size                   16588 non-null  object                               
 7   created_date           16602 non-null  datetime64[ns, pytz.FixedOffset(540)]
 8   status                 17199 non-null  int64                      

# 재구매 전환율(두 번째 구매한 사람/ 첫 번째 구매한 사람)

In [178]:
#분모
total_paid_df = total_df[total_df['paid_date'].notnull()]
total_paid_df.shape

total_paid_sort_df = total_paid_df.sort_values(by='paid_date', ascending=False)
total_paid_sort_df.shape

total_paid_sort_dupli_df = total_paid_sort_df.drop_duplicates(subset=['user_idx'], keep='last')
total_paid_sort_dupli_df.shape

#분자

# 중복 제거하기 전에서 맨 처음 중복인 데이터를 제거하고 그 다음으로 맨 처음인 것을 중복제거 하고 진행

total_paid_sort_first_dupli_df = total_paid_sort_df[total_paid_sort_df.duplicated(subset=['user_idx'], keep='last')]

total_paid_sort_first_dupli_drop_df = total_paid_sort_df.drop(index=total_paid_sort_fist_dupli_df.index, axis=0)

total_paid_sort_first_dupli_drop_dulpi_df = total_paid_sort_first_dupli_drop_df.drop_duplicates(subset=['user_idx'], keep='last')


print("재구매 전환율 : ",total_paid_sort_first_dupli_drop_dulpi_df.shape[0]/\
      total_paid_sort_first_dupli_drop_dulpi_df.shape[0] * 100)



재구매 전환율 :  100.0
